# RNN Spelling bee challenge

In this notebook we are going to try and learn a model that can take in the pronunciation of a word as a list of phonemes, and try to spell it.  On the one hand this is easier than something like speech to text.  However one difficulty is this model will be evaluated on how well it can spell words that it has never seen before.  This is not a completely well-posed question, as there are often several reasonable spellings, and indeed, some words have [homonyms](https://en.wikipedia.org/wiki/Homonym) with different spellings.  I hope in future to try and give the model more information (e.g. country of origin), as in a real spelling bee to see if it can improve its guesses.  

## Data Stuff (not interesting)

We take our data set from [The CMU pronouncing dictionary](https://en.wikipedia.org/wiki/CMU_Pronouncing_Dictionary)

In [317]:
cmu_dict_raw = open("cmudict-0.7b").read()

first_line = "A  AH0"
last_line = "ZYWICKI  Z IH0 W IH1 K IY0"

lines = cmu_dict_raw.split("\n")

for i, l in enumerate(lines):
    if l == first_line:
        first_index = i
    if l == last_line:
        last_index = i
        
print "Example line of file : "
print lines[113108]

Example line of file : 
SPELLING  S P EH1 L IH0 NG


We get dictionaries to convert between indexes and letters/phonemes

In [3]:
phonemes = set()

for l in lines[first_index : last_index + 1]:
    word, pronounce = l.split("  ")
    for phoneme in pronounce.split():
        phonemes.add(phoneme)
        
sorted_phonemes = ["_"] + sorted(list(phonemes))

index_to_phoneme = dict(enumerate(sorted_phonemes))
phoneme_to_index = dict((v, k) for k,v in index_to_phoneme.items())

index_to_letter = dict(enumerate("_abcdefghijklmnopqrstuvwxyz"))
letter_to_index = dict((v, k) for k,v in index_to_letter.items())

In [4]:
from collections import defaultdict

pronounce_dict = {}

for l in lines[first_index : last_index + 1]:
    word, phone_list = l.split("  ")
    pronounce_dict[word.lower()] = [phoneme_to_index[p] for p in phone_list.split()]

Biggest word in dictionary

In [5]:
max_k = max([len(k) for k,v in pronounce_dict.items()])
max_v = max([len(v) for k,v in pronounce_dict.items()])
for k,v in pronounce_dict.items():
    if len(k) == max_k or  len(v) == max_v:
        print k
        print v

We get rid of words that are too long, or that have punctuation or spaces in them

In [7]:
bad_ct = set()

letters = set("abcdefghijklmnopqrstuvwxyz")
print len(pronounce_dict)
for k, v in pronounce_dict.items():
    if len(k) < 5 or len(k) > 15:
        del pronounce_dict[k]
        continue
    for c in k:
        if c not in letters:
            del pronounce_dict[k]
            break

133779
108006


Split lines into words, phonemes, convert to indexes (with padding), split into training, validation, test sets.

In [9]:
import numpy as np

pairs = np.random.permutation(list(pronounce_dict.keys()))

input_ = np.zeros((len(pairs), 16))
labels_ = np.zeros((len(pairs), 15))

for i, k in enumerate(pairs):
    v = pronounce_dict[k]
    k = k + "_" * (15 - len(k))
    v = v + [0] * (16 - len(v))
    for j, n in enumerate(v):
        input_[i][j] = n
    for j, letter in enumerate(k):
        labels_[i][j] = letter_to_index[letter]
        
input_ = input_.astype(np.int32)
labels_ = labels_.astype(np.int32)

input_test   = input_[:10000]
input_val    = input_[10000:20000]
input_train  = input_[20000:]
labels_test  = labels_[:10000]
labels_val   = labels_[10000:20000]
labels_train = labels_[20000:]

data_test  = zip(input_test, labels_test)
data_val   = zip(input_val, labels_val)
data_train = zip(input_train, labels_train)

## Tensorflow code

In [ ]:
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.models.rnn import rnn_cell, seq2seq

This cell resets the graphs and session

In [337]:
ops.reset_default_graph()
try:
    sess.close()
except:
    
    pass
sess = tf.InteractiveSession()

In [338]:
input_seq_length = 16
output_seq_length = 15
batch_size = 128

input_vocab_size = 70
output_vocab_size = 28
embedding_dim = 256

As on this page we take our Seq2Seq learner to have the follwing shape:

![alt text](https://www.tensorflow.org/versions/r0.7/images/basic_seq2seq.png "Seq2Seq")

This means the decode_input has to be shifted along by one from the labels

In [339]:
encode_input = [tf.placeholder(tf.int32, 
                                shape=(None,),
                                name = "ei_%i" %i)
                                for i in range(input_seq_length)]

labels = [tf.placeholder(tf.int32,
                                shape=(None,),
                                name = "l_%i" %i)
                                for i in range(output_seq_length)]

decode_input = [tf.zeros_like(encode_input[0], dtype=np.int32, name="GO")] + labels[:-1]

This cell is the meat of the model, and a lot is happening here under the hood.  We take our cells to be LSTM recurrent units, with dropout between the feed-forward layers.  We take 3 of these stacked as our neural network.  We then run this using the seq2seq.embedding_rnn_seq2seq pattern - this let's us hand the neural network sequences like 1,2,3,2,1 - and the neural network automatically embeds this as a one-hot tensor for us.  

Note that we build two networks within the 'decoders' scope.  One of these is using feed_previous = True, the other not.  We set this to False during training, so that even if the learner makes a mistake on a letter - we still give it the correct label in the decoder_inputs.  Since we don't have the real label for the test set, this is set to True, and the decoder takes the letter with maximum probability from the last step of the decoder output.  

The decode_output is a tensor of shape (batch_size, output_vocab_size).  We can run softmax on this to get logit scores for each letter.

In [340]:
cells = [rnn_cell.DropoutWrapper(
        rnn_cell.BasicLSTMCell(embedding_dim), output_keep_prob=0.5
    ) for i in range(3)]

stacked_lstm = rnn_cell.MultiRNNCell(cells)

with tf.variable_scope("decoders") as scope:
    decode_outputs, decode_state = seq2seq.embedding_rnn_seq2seq(
        encode_input, decode_input, cell, input_vocab_size, output_vocab_size)
    
    scope.reuse_variables()
    
    decode_outputs_test, decode_state_test = seq2seq.embedding_rnn_seq2seq(
        encode_input, decode_input, cell, input_vocab_size, output_vocab_size, 
    feed_previous=True)

sequence_loss is cross-entropy on the soft max of the decode outputs.

In [341]:
loss_weights = [tf.ones_like(l, dtype=tf.float32) for l in labels]
loss = seq2seq.sequence_loss(decode_outputs, labels, loss_weights, output_vocab_size)
optimizer = tf.train.AdamOptimizer(1e-4)
train_op = optimizer.minimize(loss)

In [342]:
sess.run(tf.initialize_all_variables())

## Training model

Simple class for getting random batches and reshaping them properly for the model.

In [343]:
class DataIterator:
    def __init__(self, data, batch_size):
        self.data = data
        self.batch_size = batch_size
        self.iter = self.make_random_iter()
        
    def next_batch(self):
        try:
            idxs = self.iter.next()
        except StopIteration:
            self.iter = self.make_random_iter()
            idxs = self.iter.next()
        X, Y = zip(*[self.data[i] for i in idxs])
        X = np.array(X).T
        Y = np.array(Y).T
        return X, Y

    def make_random_iter(self):
        splits = np.arange(self.batch_size, len(self.data), self.batch_size)
        it = np.split(np.random.permutation(range(len(self.data))), splits)[:-1]
        return iter(it)
    
train_iter = DataIterator(data_train, 128)
val_iter = DataIterator(data_val, 128)
test_iter = DataIterator(data_test, 128)

Our evaluation scores are based on the seq2seq loss, and on the precision - the number of words that the model spells perfectly.

In [346]:
import sys

def get_feed(X, Y):
    feed_dict = {encode_input[t]: X[t] for t in range(input_seq_length)}
    feed_dict.update({labels[t]: Y[t] for t in range(output_seq_length)})
    return feed_dict

def train_batch(data_iter):
    X, Y = data_iter.next_batch()
    feed_dict = get_feed(X, Y)
    _, out = sess.run([train_op, loss], feed_dict)
    return out

def get_eval_batch_data(data_iter):
    X, Y = data_iter.next_batch()
    feed_dict = get_feed(X, Y)
    all_output = sess.run([loss] + decode_outputs_test, feed_dict)
    eval_loss = all_output[0]
    decode_output = np.array(all_output[1:]).transpose([1,0,2])
    return eval_loss, decode_output, X, Y

def eval_batch(data_iter, num_batches):
    losses = []
    predict_loss = []
    for i in range(num_batches):
        eval_loss, output, X, Y = get_eval_batch_data(data_iter)
        losses.append(eval_loss)
        
        for index in range(len(output)):
            real = Y.T[index]
            predict = np.argmax(output, axis = 2)[index]
            predict_loss.append(all(real==predict))
    return np.mean(losses), np.mean(predict_loss)

I simply ran this until it looked like the validation set score was not improving then aborted with a keyboard interrupt.  This took about 20 minutes on my Titan X

In [348]:
for i in range(10000):
    try:
        train_batch(train_iter)
        if i % 1000 == 0:
            val_loss, val_predict = eval_batch(val_iter, 16)
            train_loss, train_predict = eval_batch(train_iter, 16)
            print "val loss   : %f, val predict   = %.1f%%" %(val_loss, val_predict * 100)
            print "train loss : %f, train predict = %.1f%%" %(train_loss, train_predict * 100)
            print
            sys.stdout.flush()
    except KeyboardInterrupt:
        print "interrupted by user"
        break

val loss : 0.200953, val predict = 40.0%
train loss : 0.165016, train predict = 48.0%

val loss : 0.187188, val predict = 42.5%
train loss : 0.143054, train predict = 50.6%

val loss : 0.175085, val predict = 45.8%
train loss : 0.115830, train predict = 56.4%

val loss : 0.178171, val predict = 45.7%
train loss : 0.111954, train predict = 58.9%

val loss : 0.172554, val predict = 49.3%
train loss : 0.091568, train predict = 65.0%

val loss : 0.181853, val predict = 46.6%
train loss : 0.083485, train predict = 66.0%

val loss : 0.179023, val predict = 48.3%
train loss : 0.064898, train predict = 72.2%

interrupted by user


## Examining model outputs

We reach ~ 50% on our hold-out validation set, which may seem low.  Let's see some of the outputs on our test data set, to see the kind of mistakes that the model is making

In [ ]:
eval_loss, output, X, Y = get_eval_batch_data(test_iter)

In [371]:
print "pronunciation".ljust(40),
print "real spelling".ljust(17),
print "model spelling".ljust(17),
print "is correct"
print

for index in range(len(output)):
    phonemes = "-".join([index_to_phoneme[p] for p in X.T[index]]) 
    real = [index_to_letter[l] for l in Y.T[index]] 
    predict = [index_to_letter[l] for l in np.argmax(output, axis = 2)[index]]
   
    print phonemes.split("-_")[0].ljust(40),
    print "".join(real).split("_")[0].ljust(17),
    print "".join(predict).split("_")[0].ljust(17),
    print str(real == predict)

pronunciation                            real spelling     model spelling    is correct

HH-EH1-P-N-ER0                           hepner            hepner            True
P-AA1-M-D-EY2-L                          palmdale          pomdale           False
B-AY1-S-ER0                              beiser            beiser            True
G-W-AA1-V-AH0                            guava             guava             True
M-AO1-N-D-ER0-IH0-NG-Z                   maunderings       monderings        False
R-IH1-M-AH0-L                            rimel             rimmel            False
SH-UW0-V-R-AA1-N-T                       cheuvront         shovrant          False
L-IH1-Z-AH0-K                            lizak             lizac             False
B-R-AY1-B-Z                              bribes            bribes            True
K-AE1-L-AH0-B-ER0                        caliber           caliber           True
W-IY2-D-ER0-AO1-F-B-AW0                  wiederaufbau      weiderafbu        False
JH-